<a href="https://colab.research.google.com/github/Gilnanderson/assistente-virtual/blob/main/Assistente_virtual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install gTTS

from gtts import gTTS

text_to_say = "Fala, gatinha! Tudo bem?"

language = "pt"

gtts_object = gTTS(text = text_to_say,
                  lang = language,
                  tld='com.br',
                  slow = False)

gtts_object.save("/content/gtts.wav")

from IPython.display import Audio

Audio("/content/gtts.wav")

# french_text = "Je vais au supermarché"

# french_language = "fr"

# french_gtts_object = gTTS(text = french_text,
#                           lang = french_language,
#                           slow = True)

# french_gtts_object.save("/content/french.wav")

# Audio("/content/french.wav")

In [ ]:
# Instalação dos pacotes necessários
!pip install SpeechRecognition section gTTS pyjokes wikipedia pygame requests bs4
!apt-get install ffmpeg

import section
import speech_recognition as sr
from gtts import gTTS
import os
from datetime import datetime
import pyjokes
import wikipedia
import webbrowser  # já incluso no Python
from pygame import mixer  # playback limitado no Colab

from google.colab import output
from IPython.display import Javascript, display, Audio, HTML
import base64
import requests
from bs4 import BeautifulSoup
import time  # para inserção de delay

# Código JavaScript para gravar áudio em formato WebM
RECORD_AUDIO_JS = """
async function recordAudio(sec) {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const mediaRecorder = new MediaRecorder(stream);
  let audioChunks = [];
  mediaRecorder.ondataavailable = function(e) {
    audioChunks.push(e.data);
  };
  mediaRecorder.start();
  await new Promise(resolve => setTimeout(resolve, sec * 1000));
  mediaRecorder.stop();
  await new Promise(resolve => mediaRecorder.onstop = resolve);
  const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });
  const reader = new FileReader();
  reader.readAsDataURL(audioBlob);
  return new Promise(resolve => {
    reader.onloadend = () => resolve(reader.result);
  });
}
"""

def record_audio(sec=5):
    # Exibe o código JavaScript para gravar áudio e retorna os bytes do áudio
    display(Javascript(RECORD_AUDIO_JS))
    audio_data = output.eval_js(f'recordAudio({sec})')
    header, encoded = audio_data.split(',', 1)
    audio_bytes = base64.b64decode(encoded)
    return audio_bytes

def get_audio(prompt=True, suppress_error=False):
    """
    Grava áudio, converte para WAV e retorna o texto reconhecido.

    Parâmetros:
      - prompt (bool): se True, exibe mensagem informando que a gravação vai iniciar.
      - suppress_error (bool): se True, não reproduz mensagens de erro (útil na espera pela palavra de ativação).
    """
    if prompt:
        print("Gravando áudio por 5 segundos... Fale algo!")
    audio_bytes = record_audio(sec=5)
    with open("recorded.webm", "wb") as f:
        f.write(audio_bytes)
    # Converte o arquivo webm para WAV usando ffmpeg
    os.system("ffmpeg -y -i recorded.webm recorded.wav")

    r = sr.Recognizer()
    with sr.AudioFile("recorded.wav") as source:
        audio = r.record(source)
    said = ""
    try:
        said = r.recognize_google(audio, language="en-US")
        print("Você disse:", said)
    except sr.UnknownValueError:
        if not suppress_error:
            speak("Sorry, I did not get that.")
    except sr.RequestError:
        if not suppress_error:
            speak("Sorry, the service is not available")
    return said.lower()

def speak(text):
    # Converte o texto em áudio usando gTTS e exibe um widget de áudio no Colab
    tts = gTTS(text=text, lang='en')
    filename = "voice.mp3"
    if os.path.exists(filename):
        os.remove(filename)
    tts.save(filename)
    display(Audio(filename, autoplay=True))

def playmusic(song):
    # Exibe widget de áudio para reproduzir o arquivo de música
    display(Audio(song, autoplay=True))

def stopmusic():
    # Informa que a função de parar música é limitada no ambiente Colab
    print("Stopping playback (functionality may be limited in Colab)")

def respond(text):
    # Processa o comando reconhecido e executa a ação correspondente.
    print("Texto reconhecido:", text)

    # Comando para sair do programa
    if 'exit' in text:
        speak("Goodbye, till next time.")
        return True  # Sinaliza para encerrar o loop

    elif 'youtube' in text:
        speak("What do you want to search for on YouTube?")
        # Aguarda o término do TTS para evitar captar o áudio da mensagem
        time.sleep(3)
        keyword = get_audio(prompt=False)
        if keyword != '':
            search_youtube(keyword)

    elif 'search' in text:
        speak("What do you want to search for on Wikipedia?")
        query = get_audio(prompt=False)
        if query != '':
            try:
                result = wikipedia.summary(query, sentences=3)
                speak("According to Wikipedia")
                print(result)
                speak(result)
            except Exception as e:
                speak("Sorry, I couldn't fetch the Wikipedia summary.")

    elif 'joke' in text:
        joke = pyjokes.get_joke()
        speak(joke)

    elif 'empty recycle bin' in text:
        speak("Empty recycle bin command is not supported in this environment.")

    elif 'what time' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)

    elif 'play music' in text or 'play song' in text:
        speak("Now playing music...")
        music_dir = "/content/"
        songs = [f for f in os.listdir(music_dir) if f.endswith('.mp3')]
        if songs:
            print("Músicas disponíveis:", songs)
            playmusic(os.path.join(music_dir, songs[0]))
        else:
            speak("No music files found in the directory.")

    elif 'stop music' in text:
        speak("Stopping playback.")
        stopmusic()

    return False  # Sinaliza para continuar o loop

def wait_for_wake_word():
    # Aguarda até que o usuário diga a palavra de ativação "computer" sem exibir mensagens repetitivas
    print("Aguardando a palavra de ativação 'computer'...")
    while True:
        wake_text = get_audio(prompt=False, suppress_error=True)
        if "computer" in wake_text:
            print("Palavra de ativação reconhecida.")
            break
        # Se não detectar, apenas aguarda sem imprimir mensagens adicionais

def search_youtube(keyword):
    # Função para buscar e abrir o primeiro vídeo do YouTube
    search_url = f"https://www.youtube.com/results?search_query={keyword}"
    response = requests.get(search_url)

    # Verifica se a resposta foi bem-sucedida
    if response.status_code != 200:
        speak("Sorry, there was an error with the request.")
        return

    # Usando o BeautifulSoup para fazer o parsing da página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrando o primeiro vídeo com base no ID do vídeo no YouTube
    video_url = None
    for script_tag in soup.find_all('script'):
        if 'var ytInitialData =' in str(script_tag):
            json_data = str(script_tag).split('var ytInitialData =')[1].split(';</script>')[0]
            # Encontrando o primeiro vídeo na lista
            start_index = json_data.find('watch?v=')
            if start_index != -1:
                video_url = 'https://www.youtube.com/' + json_data[start_index:start_index+19]
                break

    if video_url:
        # Abre a página do primeiro vídeo usando JavaScript
        display(HTML(f'<script>window.open("{video_url}", "_blank")</script>'))
        speak(f"Opening the first video for {keyword} on YouTube")
    else:
        speak("Sorry, no video found.")

# Loop principal do programa
while True:
    wait_for_wake_word()  # Aguarda a palavra de ativação
    print("Estou ouvindo o comando...")
    command_text = get_audio()  # Captura o comando após a ativação
    if respond(command_text):  # Se o comando for 'exit', encerra o loop
        break


In [20]:
# Instalação dos pacotes necessários
!pip install SpeechRecognition section gTTS pyjokes wikipedia pygame requests bs4
!apt-get install ffmpeg

import section
import speech_recognition as sr
from gtts import gTTS
import os
from datetime import datetime
import pyjokes
import wikipedia
import webbrowser  # já incluso no Python
from pygame import mixer  # playback limitado no Colab

from google.colab import output
from IPython.display import Javascript, display, Audio, HTML
import base64
import requests
from bs4 import BeautifulSoup
import time  # para inserção de delay

# Define o idioma do Wikipedia para português
wikipedia.set_lang("pt")

# Código JavaScript para gravar áudio em formato WebM
RECORD_AUDIO_JS = """
async function recordAudio(sec) {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const mediaRecorder = new MediaRecorder(stream);
  let audioChunks = [];
  mediaRecorder.ondataavailable = function(e) {
    audioChunks.push(e.data);
  };
  mediaRecorder.start();
  await new Promise(resolve => setTimeout(resolve, sec * 1000));
  mediaRecorder.stop();
  await new Promise(resolve => mediaRecorder.onstop = resolve);
  const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });
  const reader = new FileReader();
  reader.readAsDataURL(audioBlob);
  return new Promise(resolve => {
    reader.onloadend = () => resolve(reader.result);
  });
}
"""

def record_audio(sec=5):
    # Exibe o código JavaScript para gravar áudio e retorna os bytes do áudio
    display(Javascript(RECORD_AUDIO_JS))
    audio_data = output.eval_js(f'recordAudio({sec})')
    header, encoded = audio_data.split(',', 1)
    audio_bytes = base64.b64decode(encoded)
    return audio_bytes

def get_audio(prompt=True, suppress_error=False):
    """
    Grava áudio, converte para WAV e retorna o texto reconhecido.

    Parâmetros:
      - prompt (bool): se True, exibe mensagem informando que a gravação vai iniciar.
      - suppress_error (bool): se True, não reproduz mensagens de erro (útil na espera pela palavra de ativação).
    """
    if prompt:
        print("Gravando áudio por 5 segundos... Fale algo!")
    audio_bytes = record_audio(sec=5)
    with open("recorded.webm", "wb") as f:
        f.write(audio_bytes)
    # Converte o arquivo webm para WAV usando ffmpeg
    os.system("ffmpeg -y -i recorded.webm recorded.wav")

    r = sr.Recognizer()
    with sr.AudioFile("recorded.wav") as source:
        audio = r.record(source)
    said = ""
    try:
        said = r.recognize_google(audio, language="pt-BR")
        print("Você disse:", said)
    except sr.UnknownValueError:
        if not suppress_error:
            speak("Desculpe, não entendi.")
    except sr.RequestError:
        if not suppress_error:
            speak("Desculpe, serviço não disponível.")
    return said.lower()

def speak(text):
    # Converte o texto em áudio usando gTTS e exibe um widget de áudio no Colab
    tts = gTTS(text=text, lang='pt')
    filename = "voice.mp3"
    if os.path.exists(filename):
        os.remove(filename)
    tts.save(filename)
    display(Audio(filename, autoplay=True))

def playmusic(song):
    # Exibe widget de áudio para reproduzir o arquivo de música
    display(Audio(song, autoplay=True))

def stopmusic():
    # Informa que a função de parar música é limitada no ambiente Colab
    print("Stopping playback (functionality may be limited in Colab)")

def respond(text):
    # Processa o comando reconhecido e executa a ação correspondente.
    print("Texto reconhecido:", text)

    # Comando para sair do programa
    if 'exit' in text or 'sair' in text:
        speak("Tchau, até a próxima.")
        return True  # Sinaliza para encerrar o loop

    elif 'youtube' in text:
        speak("O que você quer pesquisar no YouTube?")
        # Aguarda o término do TTS para evitar captar o áudio da mensagem
        time.sleep(3)
        keyword = get_audio(prompt=False)
        if keyword != '':
            search_youtube(keyword)

    elif 'pesquisar' in text:
        speak("O que você quer pesquisar no Wikipedia?")
        query = get_audio(prompt=False)
        if query != '':
            try:
                result = wikipedia.summary(query, sentences=3)
                speak("De acordo com o Wikipedia")
                print(result)
                speak(result)
            except wikipedia.exceptions.DisambiguationError:
                speak("Sua pesquisa é ambígua. Por favor, seja mais específico.")
            except wikipedia.exceptions.PageError:
                speak("Não foi encontrada uma página correspondente à sua pesquisa.")
            except Exception as e:
                speak("Desculpe, não foi possível fazer a busca no Wikipedia.")

    elif 'joke' in text or 'piada' in text:
        joke = pyjokes.get_joke(language='pt', category='all')
        speak(joke)

    elif 'empty recycle bin' in text:
        speak("O comando para esvaziar a lixeira não é suportado neste ambiente.")

    elif 'what time' in text or 'horas' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)

    elif 'play music' in text or 'tocar música' in text:
        speak("Tocando música...")
        music_dir = "/content/"
        songs = [f for f in os.listdir(music_dir) if f.endswith('.mp3')]
        if songs:
            print("Músicas disponíveis:", songs)
            playmusic(os.path.join(music_dir, songs[0]))
        else:
            speak("Nenhum arquivo de música encontrado no diretório.")

    elif 'stop music' in text or 'parar música' in text:
        speak("Parando a reprodução.")
        stopmusic()

    return False  # Sinaliza para continuar o loop

def wait_for_wake_word():
    # Aguarda até que o usuário diga a palavra de ativação "computador" sem exibir mensagens repetitivas
    print("Aguardando a palavra de ativação 'computador'...")
    while True:
        wake_text = get_audio(prompt=False, suppress_error=True)
        if "computador" in wake_text:
            print("Palavra de ativação reconhecida.")
            break

def search_youtube(keyword):
    # Função para buscar e abrir o primeiro vídeo do YouTube
    search_url = f"https://www.youtube.com/results?search_query={keyword}"
    response = requests.get(search_url)

    # Verifica se a resposta foi bem-sucedida
    if response.status_code != 200:
        speak("Desculpe, houve um erro com a solicitação.")
        return

    # Usando o BeautifulSoup para fazer o parsing da página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrando o primeiro vídeo com base no ID do vídeo no YouTube
    video_url = None
    for script_tag in soup.find_all('script'):
        if 'var ytInitialData =' in str(script_tag):
            json_data = str(script_tag).split('var ytInitialData =')[1].split(';</script>')[0]
            # Encontrando o primeiro vídeo na lista
            start_index = json_data.find('watch?v=')
            if start_index != -1:
                video_url = 'https://www.youtube.com/' + json_data[start_index:start_index+19]
                break

    if video_url:
        # Abre a página do primeiro vídeo usando JavaScript
        display(HTML(f'<script>window.open("{video_url}", "_blank")</script>'))
        speak(f"Abrindo o vídeo de {keyword} no YouTube")
    else:
        speak("Desculpe, vídeo não encontrado.")

# Loop principal do programa
while True:
    wait_for_wake_word()  # Aguarda a palavra de ativação
    print("Estou ouvindo o comando...")
    command_text = get_audio()  # Captura o comando após a ativação
    if respond(command_text):  # Se o comando for 'exit' ou 'sair', encerra o loop
        break


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Aguardando a palavra de ativação 'computador'...


<IPython.core.display.Javascript object>

Você disse: computador
Palavra de ativação reconhecida.
Estou ouvindo o comando...
Gravando áudio por 5 segundos... Fale algo!


<IPython.core.display.Javascript object>

Você disse: pesquisar
Texto reconhecido: pesquisar


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Você disse: espera um pouquinho já já vamos dar um computador
Palavra de ativação reconhecida.
Estou ouvindo o comando...
Gravando áudio por 5 segundos... Fale algo!


<IPython.core.display.Javascript object>

Você disse: sair
Texto reconhecido: sair
